# **PASOS Y SUGERENCIAS**

## *PASOS*
1. Conectarse a google colab
2. Ejecutar primera celda para conectarse al google drive institucional
3. Definir rootPath, savePath y ejecutar celda
4. Especificar la carpeta por aumentar, ejecutar esa celda y asegurar de que no de error
5. Ejecutar la celda MAIN SCRIPT y esperar a que se termine de ejecutar (~45s/MB sin contar al tiempo que requieren las rotaciones)
6. Definir la carpeta original, la carpeta aumentada, ejecutar ultima celda y revisar que los videos se hayan aumentado x10
7. Entrar a "Entorno de ejecucion" y "Restablecer la configuracion de fabrica del entorno de ejecucion"
8. Repetir pasos 4-5-6-7-8 para otra carpeta

TROUBLESHOOTING: En caso de que en el paso 6 vemos que no se aumento exactamente x10:

* (Solucion mas facil para nosotros) Usar la celda 'opcional' para borrar la carpeta aumentada, y repetir pasos 7-5-6 para aumentarla de nuevo
* (Solucion mas rapida para ejecutar) NO borrar la carpeta aumentada, si no identificar los videos ORIGINALES que no se aumentaron x10, copiarlos a una carpeta temporal, repetir pasos 4-5-6 para aumentar esa carpeta temporal, y mover los videos aumentados para completar la carpeta que no se aumento bien desde un inicio

## *SUGERENCIAS*
* Usar el Drive institucional por su almacenamiento ilimitado
* Asegurar de que tienes acceso de 'editor' a la carpeta que quieres aumentar
* Por mas pesada que es la carpeta, mas tiempo de demorara en aumentar. Para la primera vez, aumentar la carpeta mas liviana para asegurar de que se entienda el proceso en el menor tiempo posible
* Tener cuidado con los espacios que hay en los nombres de las carpetas
* Poner un \ antes de cualquier espacio en tu Path cuando el Path NO esta dentro de comillas.
* A veces el ambiente de colab se desconecta de nuestro Drive. Si se produce algun error inexplicable, intentar 'Conectarse a Google Drive' para arreglarlo
* 'Restablecer la configuracion de fabrica del entorno de ejecucion' despues de aumentar cada carpeta
* Despues de aumentar todas las carpetas necesarias, borrar la carpeta 'save_frames' directamente desde el Drive

# **CODIGO PARA MULTIPLICAR UNA CARPETA DE VIDEOS X10**

##Conectarse a Google Drive

In [ ]:
from google.colab import drive                                  # De preferencia conectense al Drive con el correo institucional
drive.mount('/content/gdrive', force_remount=True)              # ya que ahi tienen almacenamiento ilimitado

Mounted at /content/gdrive


##Define imports, functions, rootPath and savePath

In [ ]:
from glob import glob
import cv2 as cv
import numpy as np
import os
from datetime import datetime
import shutil
from moviepy.editor import *

rootPath = '/content/gdrive/MyDrive/TALOV Internship/Data_testing/Data_Accept'         # Directorio de la carpeta madre en el Drive
savePath = '/content/gdrive/MyDrive/TALOV Internship/Data_testing/Data_Accept_Augmented_1' # Nombre que le quieres dar a la nueva carpeta madre aumentada

                                                                    # EJEMPLOS DE ROOTPATH Y SAVEPATH
# rootPath = '/content/gdrive/MyDrive/Data_testing/ASL'             # Carpeta debe contener subcarpetas, cada subcarpeta con 60 videos
# savePath = '/content/gdrive/MyDrive/Data_testing/ASL_Augmented'   # Nueva carpeta para contener subcarpetas aumentadas, cada subcarpeta con 600 videos

print('\nRoot path:       ', rootPath)
print('Save path:       ', savePath)

# Funcion para ver si existen la carpeta por ser aumentada y/o la carpeta aumentada
def checkFolder():
  flag = 0
  for i in os.listdir(rootPath):
    if i==datasetFolder:
      print('Carpeta esta lista para aumentar x10 ==>       ', datasetFolder)
      flag = 1
      try:
        for j in os.listdir(savePath):
          if j==datasetFolder:   print('ERROR 1: ya existe una carpeta aumentada con ese nombre. Asegurar de que esa carpta este bien aumentada, caso contrario borrarla y aumentarla de nuevo.')
      except:   print('Una nueva carpeta madre aumentada sera creada al ejecutar el MAIN SCRIPT por primera vez')
  if flag == 0:
    print('ERROR 2: No se encuentra la carpeta ==>       ', datasetFolder)
    print('         Asegurar de que exista y este bien escrita.')

# Funcion para enlistar todas las carpetas en nuestras carpetas madres (original y aumentada)
def listWords():
  print('----------------Lista de todas las palabras en la carpeta madre.----------------\n', os.listdir(rootPath))
  try: print('\n--------------Lista de todas las palabras en la carpeta aumentada.--------------\n', os.listdir(savePath))
  except: print('\nLa carpeta madre aumentada aun no existe, se creara automaticamente despues de ejecutar el "Main Script" por primera vez.\n')

# Funcion para borrar una carpeta aumentada
def borrarCarpeta(borraEstaPalabra):
  print('Wait. Removing excess files . . .')
  try:
    shutil.rmtree(savePath + '/save_frames/' + borraEstaPalabra +'_frames')    # borra carpeta de frames
    print('"'+borraEstaPalabra + '_frames" fue borrado exitosamente.')
  except: print('La carpeta "'+ borraEstaPalabra +'_frames" no existe')
  try:
    shutil.rmtree(savePath + '/'+ borraEstaPalabra)                            # borra carpeta aumentada
    print('"'+borraEstaPalabra + '" fue borrado exitosamente.')
  except: print('La carpeta "'+ borraEstaPalabra +'" no existe')

# Funcion para leer un video y guardar todas sus frames
def save_frame(video_path, save_dir, save_name):
  save_path = os.path.join(save_dir, save_name)
  create_dir(save_path)
  cap = cv.VideoCapture(video_path)
  idx = 0
  while True:
    ret, frame = cap.read()
    if ret == False:
      cap.release()
      break
    cv.imwrite(f"{save_path}/{idx}.jpg", frame)
    idx += 1
  print('--')
  print(save_name+" video was split in frames.")
  frame_array = glob(save_dir+"/"+save_name+"/*")
  return frame_array

# Funcion para crear un directorio si es que no existe
def create_dir(path):
  try:
    if not os.path.exists(path):
      os.makedirs(path)
  except OSError:
    print(f"ERROR: creating directory with name {path}")

# Funcion para crear 9 variaciones de un frame cambiando su valor de iluminacion
def augmentation(frame_array, save_name):
  for frame_dir in frame_array:
    img = cv.imread(frame_dir, cv.IMREAD_UNCHANGED)
    ilu1 = adjust_gamma(img, 0.50)                     # ilumination low
    ilu2 = adjust_gamma(img, 1.50)                     # ilumination high
    ilu3 = adjust_gamma(img, 2.00)
    ilu4 = adjust_gamma(img, 2.50)
    ilu5 = adjust_gamma(img, 3.00)
    ilu6 = adjust_gamma(img, 3.50)
    ilu7 = adjust_gamma(img, 4.00)
    ilu8 = adjust_gamma(img, 4.50)
    ilu9 = adjust_gamma(img, 5.00)
    aug_imgs = [ilu1, ilu2, ilu3, ilu4, ilu5, ilu6, ilu7, ilu8, ilu9]
    for i in range (1,10):
      new_save_name = str(frame_dir)[:len(str(frame_dir))-4] + '_mod_' + str(i) + '.jpg'
      cv.imwrite(new_save_name,aug_imgs[i-1])
  print(save_name +" frames were augmented.")

# Funcion para modificar la iluminacion de un frame
def adjust_gamma(image, gamma):
  invGamma = 1.0 / gamma
  table = np.array([((i / 255.0) ** invGamma) * 255
    for i in np.arange(0, 256)]).astype("uint8")
  return cv.LUT(image, table)

# Funcion para leer conjuntos de frames aumentadas y crear nuevos videos
def createVideos(word, save_name, save_dir_frame, fps, Hflag):
  horizontal_paths = []
  print('.Video mods are being created...')
  for i in range (1,10):                                                # corre para cada uno de los 9 conjuntos de frames modificados
    create_dir(savePath + "/" + word)
    aug_vid_name = savePath + "/" + word + "/" + save_name + "_augmented_" + str(i) + '.mp4'
    horizontal_paths.append(aug_vid_name)
    aug_frame_array = glob(save_dir_frame+"/"+save_name+"/"+"*_mod_"+str(i)+"*")# crea una lista de paths para un conjunto de frames modificados

    img_array = []
    for filename in aug_frame_array:                                            # lee cada frame de esa lista de paths y lo agrega a un array
      img = cv.imread(filename)
      height, width, layers = img.shape
      size = (width,height)
      img_array.append(img)

    out = cv.VideoWriter(aug_vid_name, cv.VideoWriter_fourcc(*'DIVX'), fps,size)# convierte el array de frames modificados a un nuevo video

    for j in range(len(img_array)):
      out.write(img_array[j])
    out.release()
  print('.Video mods were created.')

  if (Hflag==1):
    print(".Rotating horizontal augmented videos now...")
    for path in horizontal_paths:
      save_path = path
      myvideo = VideoFileClip(path)
      os.remove(path)
      myvideo = myvideo.rotate(-90)
      myvideo.write_videofile(save_path)

# Funcion para determinar si un video tiene dimensiones horizontales
def is_Horizontal(path, Hflag):
  vid = cv.VideoCapture(path)
  fps = vid.get(cv.CAP_PROP_FPS)
  width = vid.get(3)
  height = vid.get(4)
  vid.release()
  if height < width:
    Hflag = 1
  return Hflag, fps

Imageio: 'ffmpeg-linux64-v3.3.1' was not found on your computer; downloading it now.
Try 1. Download from https://github.com/imageio/imageio-binaries/raw/master/ffmpeg/ffmpeg-linux64-v3.3.1 (43.8 MB)
Downloading: 8192/45929032 bytes (0.0%)3366912/45929032 bytes (7.3%)6602752/45929032 bytes (14.4%)9756672/45929032 bytes (21.2%)13017088/45929032 bytes (28.3%)16064512/45929032 bytes (35.0%)18874368/45929032 bytes (41.1%)21921792/45929032 bytes (47.7%)25092096/45929032 bytes (54.6%)28336128/45929032 bytes (61.7%)31481856/45929032 bytes (68.5%)34725888/45929032 bytes (75.6%)37847040/45929032 bytes (82.4%)

## (Opcional) Correr si deseas visualizar:

In [ ]:
# LISTA DE TODO EL CONTENIDO DENTRO DE UNA CARPETA (con sus respectivo tamaños)
! du -h -a --max-depth=1    /content/gdrive/MyDrive/Sign\ Languages/United\ States
                            # aqui va la carpeta madre (rootPath)

In [ ]:
# EL TAMAÑO DE UN SOLO ARCHIVO/CARPETA/VIDEO
! du -sh                /content/gdrive/MyDrive/Sign\ Languages/United\ States/Great\ granddaughter
                        # aqui va cualquier carpeta o documento

In [ ]:
# LISTAS DE TODAS LAS PALABRAS EN LA CARPETA MADRE Y LA NUEVA CARPETA AUMENTADA
listWords()

----------------Lista de todas las palabras en la carpeta madre.----------------
 ['Allergy', 'Anatomy', 'Fine', 'Asphyxiate', 'Ambulance', 'Few', 'Again', 'Audiology', 'Abuse', 'Also', 'Ask', 'Alarm', 'Asthma', 'Alcohol', 'Alcoholic Anonymus', 'Addiction', 'Heartburn', 'Mental health', 'Pill', 'Help', 'Pneumonia', 'Outside', 'Never', 'He', 'Operate', 'Medicine', 'Get', 'Hard', 'Month', 'Pap smear', 'Prefer', 'Learn', 'Home', 'Open', 'Hospital', 'Now', 'Next', 'Hot', 'Offhand', 'For', 'Interpreter', 'Love', 'Inject', 'Nearby', 'Inform', 'Injure', 'Make', 'Mount', 'Rape', 'Keep', 'Read', 'Good', 'House', 'Hearing', 'Hello', 'Question', 'Have', 'Height', 'Nosebleed', 'Give', 'Morning', 'Leave', 'Menstruation', 'Picture', 'Only', 'Pregnant', 'Joint', 'Meaning', 'Hurt', 'Night', 'Prescription', 'Must', 'Run', 'Tomorrow', 'Today', 'Study', 'Try', 'They', 'Some', 'Time', 'Think', 'Until', 'Understand', 'Still', 'Visit', 'Swollen', 'Start', 'Their', 'Therapist', 'Wait', 'True', 'Sprain', 'Sit

##(Opcional) Para borrar carpetas

In [ ]:
# SOLO USAR si algo salio mal y quieres volver a aumentar una carpeta
# Solo se borrara la carpeta aumentada y los frames de una palabra
# NO se borrara la carpeta madre

borraEstaPalabra = 'Accept'           # especificar la carpeta aumentada que desea borrar
borrarCarpeta(borraEstaPalabra)

Wait. Removing excess files . . .
La carpeta "Accept_frames" no existe
La carpeta "Accept" no existe


## Especificar el nombre de la carpeta/palabra por aumentar


In [ ]:
datasetFolder = 'Accept'       # Definir el nombre de la carpeta por aumentar y verificar que no de error
checkFolder()

Carpeta esta lista para aumentar x10 ==>        Accept
Una nueva carpeta madre aumentada sera creada al ejecutar el MAIN SCRIPT por primera vez



##MAIN SCRIPT

In [ ]:
def main():
  folderPath = rootPath + '/' + datasetFolder
  augfolderPath = savePath + '/' + datasetFolder
  total_start_time = datetime.now()                               # toma el tiempo total
  shutil.copytree(folderPath, augfolderPath)                      # hace una copia de todos los videos originales
  print('Successfully created new Augmented', datasetFolder, 'folder with the original videos')
  video_paths = glob(augfolderPath + "/*")                        # hace una lista de los paths de todos los videos dentro de la carpeta
  save_dir_frame = savePath+'/save_frames/'+datasetFolder+'_frames'  # el path para guardar todos los frame aumentados
  idx = 0
  for path in video_paths:                                        # corre para cada video dentro de mi carpeta
    Hflag = 0
    try:
      start_time = datetime.now()                                         # toma el tiempo por video
      save_name = path.split("/")[-1].split(".")[0]
      frame_array = save_frame(path, save_dir_frame, save_name)           # 1. Guarda todos los frames de un video y hace una lista de los paths
      augmentation(frame_array, save_name)                                # 2. Augmenta los frames de un video x10
      Hflag, fps = is_Horizontal(path, Hflag)                             # 3. Revisa si el video es horizontal para despues girarlo
      createVideos(datasetFolder, save_name, save_dir_frame, fps, Hflag)     # 4. Usa los frames aumentados para crear 9 nuevos videos
      idx = idx + 1
      print('--Time elapsed for augmenting video #',idx,': ', datetime.now() - start_time)
    except: print('ERROR augmenting: ', save_name)
  print('----TOTAL TIME FOR AUGMENTING',idx,'VIDEOS IN',datasetFolder, 'FOLDER ---- ', datetime.now() - total_start_time)
  print('----READY TO AUGMENT A NEW FOLDER!')
  print('\n\n\n***Sugerencia . Reiniciar el entorno de ejecucion despues de ampliar una carpeta para \n\t\t evitar que el Disco se llene y tengas que volver a correr todo :(')
  print('\n\t\t"Entorno de ejecucion" -> "Restablecer la configuracion de fabrica del entorno de ejecucion"')

main()

Successfully created new Augmented Accept folder with the original videos
--
Accept_original_13 video was split in frames.
Accept_original_13 frames were augmented.
.Video mods are being created...
.Video mods were created.
.Rotating horizontal augmented videos now...
[MoviePy] >>>> Building video /content/gdrive/MyDrive/TALOV Internship/Data_testing/Data_Accept_Augmented_1/Accept/Accept_original_13_augmented_1.mp4
[MoviePy] Writing video /content/gdrive/MyDrive/TALOV Internship/Data_testing/Data_Accept_Augmented_1/Accept/Accept_original_13_augmented_1.mp4


100%|██████████| 33/33 [00:02<00:00, 13.79it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: /content/gdrive/MyDrive/TALOV Internship/Data_testing/Data_Accept_Augmented_1/Accept/Accept_original_13_augmented_1.mp4 

[MoviePy] >>>> Building video /content/gdrive/MyDrive/TALOV Internship/Data_testing/Data_Accept_Augmented_1/Accept/Accept_original_13_augmented_2.mp4
[MoviePy] Writing video /content/gdrive/MyDrive/TALOV Internship/Data_testing/Data_Accept_Augmented_1/Accept/Accept_original_13_augmented_2.mp4


100%|██████████| 33/33 [00:02<00:00, 13.87it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: /content/gdrive/MyDrive/TALOV Internship/Data_testing/Data_Accept_Augmented_1/Accept/Accept_original_13_augmented_2.mp4 

[MoviePy] >>>> Building video /content/gdrive/MyDrive/TALOV Internship/Data_testing/Data_Accept_Augmented_1/Accept/Accept_original_13_augmented_3.mp4
[MoviePy] Writing video /content/gdrive/MyDrive/TALOV Internship/Data_testing/Data_Accept_Augmented_1/Accept/Accept_original_13_augmented_3.mp4


...

100%|██████████| 245/245 [00:30<00:00,  8.11it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: /content/gdrive/MyDrive/TALOV Internship/Data_testing/Data_Accept_Augmented_1/Accept/Accept_whiteback_whitetshirt_glasses_augmented_5.mp4 

[MoviePy] >>>> Building video /content/gdrive/MyDrive/TALOV Internship/Data_testing/Data_Accept_Augmented_1/Accept/Accept_whiteback_whitetshirt_glasses_augmented_6.mp4
[MoviePy] Writing video /content/gdrive/MyDrive/TALOV Internship/Data_testing/Data_Accept_Augmented_1/Accept/Accept_whiteback_whitetshirt_glasses_augmented_6.mp4


100%|██████████| 245/245 [00:30<00:00,  8.14it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: /content/gdrive/MyDrive/TALOV Internship/Data_testing/Data_Accept_Augmented_1/Accept/Accept_whiteback_whitetshirt_glasses_augmented_6.mp4 

[MoviePy] >>>> Building video /content/gdrive/MyDrive/TALOV Internship/Data_testing/Data_Accept_Augmented_1/Accept/Accept_whiteback_whitetshirt_glasses_augmented_7.mp4
[MoviePy] Writing video /content/gdrive/MyDrive/TALOV Internship/Data_testing/Data_Accept_Augmented_1/Accept/Accept_whiteback_whitetshirt_glasses_augmented_7.mp4


 67%|██████▋   | 164/245 [00:18<00:11,  7.16it/s]

##Para confirmar que los 60 videos en las carpetas fueron aumentados x10 (600)

In [ ]:
! ls                /content/gdrive/MyDrive/Sign\ Languages/Original/United\ States/Labor\ day | wc -l
                    # rootPath + "/miCarpeta"

! ls               /content/gdrive/MyDrive/Data_testing/United\ States\ Augmented/Labor\ day | wc -l
                    # savePath + "/miCarpeta"

60
420
